In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [11]:
fashion_mnist = tf.keras.datasets.fashion_mnist.load_data()
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist
X_train, y_train = X_train_full[:-5000], y_train_full[:-5000]
X_valid, y_valid = X_train[-5000:], y_train_full[-5000:]

In [12]:
X_train, X_valid, X_test = X_train / 255., X_valid / 255., X_test / 255.
class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

In [61]:
tf.keras.utils.set_random_seed(42)
mode = tf.keras.Sequential([
    tf.keras.layers.Input(shape=[28,28]),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(300, activation="relu"),
    tf.keras.layers.Dense(100, activation="relu"),
    tf.keras.layers.Dense(10, activation="softmax")
])

In [102]:
cnn_model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(
        filters=64,
        kernel_size = 2,
        strides=(2,2),
        input_shape = (28,28,1),
        activation = "mish",
        kernel_initializer=tf.keras.initializers.HeNormal()),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(
        filters=64,
        kernel_size = 2,
        strides=(2,2),
        activation = "mish",
        kernel_initializer=tf.keras.initializers.HeNormal()),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(pool_size = (2,2)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(100, activation = "mish", kernel_initializer=tf.keras.initializers.HeNormal()),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(100, activation = "mish", kernel_initializer=tf.keras.initializers.HeNormal()),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10, activation="softmax")
])

In [ ]:
cnn_model.build()
cnn_model.summary()

In [106]:
cnn_model.compile(loss="sparse_categorical_crossentropy", optimizer=tf.keras.optimizers.legacy.SGD(learning_rate=0.001, momentum=0.9, nesterov=True), metrics=["accuracy"])
history = cnn_model.fit(X_train, y_train, epochs=30, validation_data=(X_valid, y_valid))

Epoch 1/30
1719/1719 [==============================] - 7s 4ms/step - loss: 0.5171 - accuracy: 0.8162 - val_loss: 9.0964 - val_accuracy: 0.1052
Epoch 2/30
1719/1719 [==============================] - 6s 4ms/step - loss: 0.5150 - accuracy: 0.8168 - val_loss: 9.3810 - val_accuracy: 0.1064
Epoch 3/30
1719/1719 [==============================] - 6s 4ms/step - loss: 0.5124 - accuracy: 0.8175 - val_loss: 8.9794 - val_accuracy: 0.1044
Epoch 4/30
1719/1719 [==============================] - 6s 3ms/step - loss: 0.5136 - accuracy: 0.8182 - val_loss: 9.3411 - val_accuracy: 0.1046
Epoch 5/30
1719/1719 [==============================] - 6s 4ms/step - loss: 0.5119 - accuracy: 0.8187 - val_loss: 9.6778 - val_accuracy: 0.1060
Epoch 6/30
1719/1719 [==============================] - 7s 4ms/step - loss: 0.5157 - accuracy: 0.8162 - val_loss: 9.1390 - val_accuracy: 0.1040
Epoch 7/30
1719/1719 [==============================] - 6s 4ms/step - loss: 0.5112 - accuracy: 0.8186 - val_loss: 9.6934 - val_accuracy:

In [107]:
cnn_model.evaluate(X_test, y_test)

313/313 [==============================] - 0s 2ms/step - loss: 0.4720 - accuracy: 0.8322


[0.4719814360141754, 0.8321999907493591]

In [7]:
datasets = tfds.load(name="mnist")
mnist_train, mnist_test = datasets["train"], datasets["test"]

for batch in mnist_train.shuffle(10_000, seed=42).batch(100).prefetch(1):
    images = batch["image"]
    labels = batch["label"]

train_x = images
train_y = labels

print(train_y)

tf.Tensor(
[9 6 7 8 2 4 2 2 7 2 3 8 0 6 5 8 1 7 4 5 8 9 8 1 4 7 2 8 4 2 5 1 2 9 1 7 3
 8 4 6 4 0 8 9 7 0 4 1 3 0 0 5 5 1 5 5 4 3 8 8 6 5 8 2 3 7 2 9 3 3 6 6 3 6
 8 2 1 2 8 9 0 8 3 8 2 5 1 3 5 7 3 9 0 3 4 7 4 6 7 3], shape=(100,), dtype=int64)


In [39]:
cnn_model_prefetched = tf.keras.Sequential([
    tf.keras.layers.Conv2D(
        filters=64,
        kernel_size = 2,
        strides=(2,2),
        input_shape = (28,28,1),
        activation = "mish",
        padding = "same",
        kernel_initializer=tf.keras.initializers.HeNormal()),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(
        filters=64,
        kernel_size = 2,
        strides=(2,2),
        activation = "mish",
        padding = "same",
        kernel_initializer=tf.keras.initializers.HeNormal()),
    tf.keras.layers.Conv2D(
        filters=64,
        kernel_size = 2,
        strides=(2,2),
        activation = "mish",
        padding = "same",
        kernel_initializer=tf.keras.initializers.HeNormal()),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(
        filters=64,
        kernel_size = 2,
        strides=(2,2),
        activation = "mish",
        padding = "same",
        kernel_initializer=tf.keras.initializers.HeNormal()),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(100, activation = "mish", kernel_initializer=tf.keras.initializers.HeNormal()),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(100, activation = "mish", kernel_initializer=tf.keras.initializers.HeNormal()),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10, activation="softmax")
])

In [41]:
cnn_model_prefetched.compile(loss="sparse_categorical_crossentropy", optimizer=tf.keras.optimizers.legacy.SGD(learning_rate=0.001, momentum=0.9, nesterov=True), metrics=["accuracy"])
history = cnn_model_prefetched.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))

Epoch 1/20
1719/1719 [==============================] - 8s 5ms/step - loss: 0.2675 - accuracy: 0.9089 - val_loss: 11.8278 - val_accuracy: 0.1000
Epoch 2/20
1719/1719 [==============================] - 8s 4ms/step - loss: 0.2628 - accuracy: 0.9090 - val_loss: 11.8520 - val_accuracy: 0.0992
Epoch 3/20
1719/1719 [==============================] - 8s 5ms/step - loss: 0.2568 - accuracy: 0.9131 - val_loss: 11.9164 - val_accuracy: 0.1016
Epoch 4/20
1719/1719 [==============================] - 8s 4ms/step - loss: 0.2538 - accuracy: 0.9126 - val_loss: 11.7571 - val_accuracy: 0.1016
Epoch 5/20
1719/1719 [==============================] - 8s 5ms/step - loss: 0.2511 - accuracy: 0.9147 - val_loss: 12.0712 - val_accuracy: 0.1004
Epoch 6/20
1719/1719 [==============================] - 8s 5ms/step - loss: 0.2451 - accuracy: 0.9160 - val_loss: 12.4177 - val_accuracy: 0.1018
Epoch 7/20
1719/1719 [==============================] - 8s 4ms/step - loss: 0.2414 - accuracy: 0.9171 - val_loss: 12.4586 - val_ac

In [42]:
cnn_model_prefetched.evaluate(X_test, y_test)

313/313 [==============================] - 1s 2ms/step - loss: 0.3193 - accuracy: 0.8986


[0.3193347752094269, 0.8985999822616577]